In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
from tqdm import tqdm

In [2]:
data_path = './Detect_solar_dust/'
dir = sorted(os.listdir(data_path))
IMG_SIZE = 300

In [19]:
data = []
bad_img = []
for i in dir:
    class_path = os.path.join(data_path, i)
    file_list = sorted(os.listdir(class_path)) 
    for f in file_list:
        # fpath = os.path.join(class_path,f)
        # img = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)
        # img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        # data.append([np.array(img), np.array(i)])
        try:
            fpath = os.path.join(class_path,f)
            img = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            data.append([img, str(i)])

        except Exception as e:
            print(str(e))
            fpath = os.path.join(class_path,f)
            bad_img.append(fpath)



# df = pd.DataFrame(data, columns=['image', 'labels'])

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



In [20]:
lenofimage = len(data)

In [21]:
X=[]
y=[]

for categories, label in data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [22]:
X = X/255.0
y=np.array(y)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [24]:
from sklearn.svm import SVC
svc = SVC(kernel='linear',gamma='auto')
svc.fit(X_train, y_train)

SVC(gamma='auto', kernel='linear')

In [25]:
y2 = svc.predict(X_test)

In [35]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
cm = confusion_matrix(y_test,y2)
print("Accuracy on unknown data is",accuracy_score(y_test,y2))
print("F1-Score on unknown data is",f1_score(y_test,y2, pos_label='Clean'))

Accuracy on unknown data is 0.7067082683307332
F1-Score on unknown data is 0.7519788918205805


In [36]:
from sklearn.metrics import classification_report
print("Accuracy on unknown data is",classification_report(y_test,y2))

Accuracy on unknown data is               precision    recall  f1-score   support

       Clean       0.76      0.75      0.75       381
       Dusty       0.64      0.65      0.64       260

    accuracy                           0.71       641
   macro avg       0.70      0.70      0.70       641
weighted avg       0.71      0.71      0.71       641



In [ ]:
result = pd.DataFrame({'original' : y_test,'predicted' : y2})
result

In [42]:
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.581 total time= 2.6min
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.578 total time= 2.6min
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.578 total time= 2.6min
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.578 total time= 2.6min
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.578 total time= 2.9min
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.581 total time= 2.9min
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.578 total time= 2.9min
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.578 total time= 3.1min
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.578 total time= 3.0min
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.578 total time= 3.0min
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.581 total time= 2.7min
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [43]:
# print best parameter after tuning
print(grid.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=10, gamma=0.0001)


In [44]:
grid_predictions = grid.predict(X_test)
  
# print classification report
print(classification_report(y_test, grid_predictions))

              precision    recall  f1-score   support

       Clean       0.78      0.81      0.79       381
       Dusty       0.70      0.66      0.68       260

    accuracy                           0.75       641
   macro avg       0.74      0.73      0.73       641
weighted avg       0.74      0.75      0.74       641



In [45]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
print("Accuracy on unknown data is",accuracy_score(y_test, grid_predictions))
print("F1-Score on unknown data is",f1_score(y_test, grid_predictions, pos_label='Clean'))

Accuracy on unknown data is 0.7457098283931357
F1-Score on unknown data is 0.7902187902187902
